In [1]:
from   astroplan import moon
from   astropy import time
from   astropy.io import ascii
from   astropy import table
import pandas as pd
import numpy as np

In [2]:
# URL to the NOT observing calendar

In [3]:
url = r'http://www.not.iac.es/observing/schedules/schedule.php?period=65&k=&output=html&prn=yes'

In [4]:
# Retrieve the calendar

In [5]:
tables = pd.read_html(url)
schedule = table.Table.from_pandas(tables[0])
schedule[:5]

Start Date (Noon),End Date (Noon),Proposal Number,Principal Investigator,Institute/ Country,Programme,Instrument,Remarks
str6,str6,str6,str54,str12,str116,str14,str61
Apr 01,Apr 02,65-026,Lauri Siltala,FI,Physical characterization of large asteroids through lightcurve inversion,ALFOSC +FIES,Time critical 5.7hr: Service
Apr 01,Apr 02,65-199,Various,NOT,Nordic service time,ALFOSC +FIES,Remainder of night: Service
Apr 02,Apr 03,64-199,Various,NOT,Nordic service time,ALFOSC +FIES,Service
Apr 03,Apr 05,65-109,Cameron Lemon,ORP,Spectroscopic Confirmation of Lensed Quasars discovered through Extended Variability,ALFOSC,--
Apr 05,Apr 06,65-299,CAT,ES,CAT service time,ALFOSC +FIES,--


In [6]:
# Identifying the relevant nights

In [7]:
mask_service   = np.where(schedule['Programme'] == 'Nordic service time')[0]
mask_technical = np.where(schedule['Programme'] == 'Technical time')[0]
mask_union     = np.union1d(mask_service, mask_technical)

In [8]:
# Return the useful observing nights

In [9]:
keys = ['Start Date (Noon)', 'End Date (Noon)', 'Instrument', 'Remarks', 'Programme']
good_nights = schedule[keys][mask_union]
good_nights[-5:]

Start Date (Noon),End Date (Noon),Instrument,Remarks,Programme
str6,str6,str14,str61,str116
Sep 19,Sep 21,ALFOSC +FIES,Remainder of nights: Service,Nordic service time
Sep 21,Sep 22,? +FIES,Service,Nordic service time
Sep 27,Sep 28,ALFOSC +FIES,Service,Nordic service time
Sep 29,Sep 30,ALFOSC +FIES,Service,Nordic service time
Sep 30,Oct 01,ALFOSC,Remainder of night,Technical time


In [10]:
# Output calendar

In [11]:
year_start = '2022'
year_end   = '2022'

months_year_start = ['Oct', 'Nov', 'Dec']
months_year_end = ['Jan', 'Feb', 'Mar', 'Apr', 'Jun', 'Jul', 'Aug', 'Sep']

month_number = {'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08', 'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

In [12]:
obs_run = table.Table(names=('Start date', 'Start Time', 'End Date', 'End Time', 'Subject'), dtype=('S100', 'S100', 'S100', 'S100', 'S1000'))

for i in range(len(good_nights)):
    
    month, day = good_nights[i]['Start Date (Noon)'].split(' ')
    date       = year_start + '-' + month_number[month] + '-' + day + 'T12:00:00' if month in months_year_start else year_end + '-' + month_number[month]  + '-' + day + 'T12:00:00'
    date       = time.Time(date, format='isot', scale='utc')
    
    tstart     = date
    
    month, day = good_nights[i]['End Date (Noon)'].split(' ')
    date       = year_start + '-' + month_number[month] + '-' + day + 'T12:00:00' if month in months_year_start else year_end + '-' + month_number[month]  + '-' + day + 'T12:00:00'
    date       = time.Time(date, format='isot', scale='utc')
    
    tend       = date
    
    moon_illum = np.round(moon.moon_illumination(tstart, ephemeris=None), 2)
    
    instrument  = good_nights[i]['Instrument']
    remarks     = good_nights[i]['Remarks']
    night_type  = good_nights[i]['Programme']
    
    description = 'Instrument(s): ' + instrument + '; moon illumination: ' + str(moon_illum)
    
    # Fetch the nights when ALFOSC is mounted

    if 'ALFOSC' in instrument:
        #print( tstart, tend, moon_illum, instrument, night_type, remarks)
        obs_run.add_row([tstart.iso.split(' ')[0], tstart.iso.split(' ')[1] + ' PM', tend.iso.split(' ')[0], tend.iso.split(' ')[1] + ' PM', night_type + ' (' + description + ')'])

In [13]:
ascii.write(obs_run, 'NOT_obsrun_P65.csv', format='csv', overwrite=True)

In [14]:
obs_run#[20:]

Start date,Start Time,End Date,End Time,Subject
bytes100,bytes100,bytes100,bytes100,bytes1000
2022-04-01,12:00:00.000 PM,2022-04-02,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.0)
2022-04-02,12:00:00.000 PM,2022-04-03,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.02)
2022-04-06,12:00:00.000 PM,2022-04-07,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.25)
2022-04-07,12:00:00.000 PM,2022-04-08,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.34)
2022-04-08,12:00:00.000 PM,2022-04-09,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.43)
2022-04-09,12:00:00.000 PM,2022-04-10,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.52)
2022-04-10,12:00:00.000 PM,2022-04-11,12:00:00.000 PM,Nordic service time (Instrument(s): ALFOSC +FIES; moon illumination: 0.62)
2022-04-11,12:00:00.000 PM,2022-04-12,12:00:00.000 PM,Technical time (Instrument(s): ALFOSC +FIES; moon illumination: 0.71)
2022-04-14,12:00:00.000 PM,2022-04-15,12:00:00.000 PM,Technical time (Instrument(s): ALFOSC +FIES; moon illumination: 0.93)
